In [1]:
from Eval_basis import *
import pandas as pd

[2019-08-10 14:06:09,135] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
np.random.seed(1)
%matplotlib inline

In [ ]:
data_full = EbiData("./data", experiment="E-ENAD-15")
data_big_mapped = UnionDataset("./data", map_fname="ensembl_mouse_genes-proteincoding", low_memory=False)
data_big_mapped.concat_union_from_memory([data_full])
data_big_mapped.filter_cell_types(np.array([ct for ct in data_big_mapped.cell_types if ct != "not available"]))

[2019-08-10 14:07:12,893] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2019-08-10 14:07:12,937] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-10 14:07:12,943] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-10 14:07:14,052] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-10 14:07:15,090] INFO - scvi.dataset.dataset | Downsampled from 50896 to 35577 cells


In [ ]:
agg = data_full.obs.groupby(["Sample Characteristic[organism part]", "cell_types"]).size()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(agg.sort_index())

In [ ]:
n_epochs = 100
colors=None
for tissue in np.unique(agg.index.get_level_values(0)): 
    print("Training VAE for tissue ", tissue)
    cutout_cts = agg[tissue].index.values
    if "not available" in cutout_cts:
        cutout_cts = np.delete(cutout_cts, np.where(cutout_cts == "not available")[0])

    rem_ct = data_big_mapped.cell_types[~np.isin(data_big_mapped.cell_types, cutout_cts)]
    data_small = copy.deepcopy(data_big_mapped)
    data_big = copy.deepcopy(data_big_mapped)

    data_small.filter_cell_types(cutout_cts)
    data_big.filter_cell_types(rem_ct)
    
    trainer_big = train_vae(data_big, "./data", f"big_{tissue}_data_portion", n_epochs=n_epochs)
    trainer_small = train_vae(data_small, "./data", f"small_{tissue}_data_portion", n_epochs=n_epochs)
    dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

    posterior_big = plot_tsne(trainer_big, trainer_big.model, data_big, f"./plots/big_{tissue}_data_portion",
                              colors=colors, s=dot_size, edgecolors='black')
    posterior_small = plot_tsne(trainer_small, trainer_small.model, data_small, f"./plots/small_{tissue}_data_portion",
                                colors=colors, s=dot_size, edgecolors='black')
    posterior_small_in_big = plot_tsne(trainer_big, trainer_big.model, data_small, f"./plots/small_{tissue}_data_portion_in_big",
                                       colors=colors, s=dot_size, edgecolors='black')